# Scaling XGBoost with Dask and Coiled

This notebook walks through training an [XGBoost](https://xgboost.readthedocs.io/en/latest/) model locally on a small dataset and then using [Dask](https://dask.org/) and [Coiled](https://coiled.io/) to scale out to the cloud and run XGBoost on a larger-than-memory dataset.

# Local XGBoost

[XGBoost](https://xgboost.readthedocs.io/en/latest/) is a popular library for training gradient boosted supervised machine learning models. 

## Load our dataset

The first step towards training our model is to load our dataset. We'll use the [Higgs dataset](https://archive.ics.uci.edu/ml/datasets/HIGGS), which is available on Amazon S3. The dataset is composed of 11 million simulated particle collisions, each of which is described by 28 real-valued, features and a binary label indicating which class the sample belongs to (i.e. whether the sample represents a signal or background event). To start, we'll load only a sample of the dataset (just over 175 thousand samples) and process the full datset in the next section.

In [ ]:
import pandas as pd

# Load a single CSV file
df = pd.read_csv("s3://coiled-data/higgs/higgs-00.csv")
df

Next, we can separate our classification label and training features and then use Scikit-learn's `sklearn.model_selection.train_test_split` function to partition the dataset into training and testing samples.

In [ ]:
X, y = df.iloc[:, 1:], df["labels"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

To use XGBoost, we'll need to construct `xgboost.DMatrix` objects for both our training and testing datasets -- these are the internal data structures XGBoost uses to manage dataset features and targets. However, since XGBoost plays well with libaries like NumPy and Pandas, we can simply pass our training and testing datasets directly to `xgboost.DMatrix(...)`.

In [ ]:
import xgboost

dtrain = xgboost.DMatrix(X_train, y_train)
dtest = xgboost.DMatrix(X_test, y_test)

Next we'll define the set of hyperparameters we want to use for our XGBoost model and train the model!

In [ ]:
params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'min_child_weight': 0.5,
}

bst = xgboost.train(params, dtrain, num_boost_round=3)

Now that our model has been trained, we can use it to make predictions on the testing dataset which was _not_ used to train the model.

In [ ]:
y_pred = bst.predict(dtest)

y_pred

To get a sense for the quality of these predictions we can compute and plot a [receiver operating characteristic (ROC) curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) of our model's predictions, which compares the predicted output from our model with the known labels to calculate the true postive rate vs. false positive rate.

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, y_pred)

In [ ]:
from sklearn.metrics import auc
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(fpr, tpr, lw=3,
        label='ROC Curve (area = {:.2f})'.format(auc(fpr, tpr)))
ax.plot([0, 1], [0, 1], "k--", lw=2)
ax.set(
    xlim=(0, 1),
    ylim=(0, 1),
    title="ROC Curve",
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
)
ax.legend()
plt.show()

# Scaling with Dask and Coiled

In the previous section, we trained a model with a subset of the full Higgs dataset. In this section, we will use the full dataset with 11 million samples! With this increased number of samples, the dataset may not fit comfortably into memory on a personal laptop. So we'll use Dask and Coiled to expand our compute resources to the cloud to enable us to work with this larger datset.

### Create a Dask cluster on AWS with Coiled

Let's create a Coiled cluster using the `examples/xgboost` software environment, which has Dask, XGBoost, Scikit-learn, and other relavant packages installed, and then connect a `dask.distributed.Client` to our cluster so we can begin to submit tasks to the cluster.

First, sign-in to Coiled Cloud (if you haven't already):
1. Go to [cloud.coiled.io](https://cloud.coiled.io)
2. Sign-in with GitHub, Google, or Email.
3. Copy your security token from the Profile page: https://cloud.coiled.io/profile (and paste it when asked below!)

In [ ]:
import coiled

cluster = coiled.Cluster(
    n_workers=10,
    software="examples/scaling-xgboost",
    protocol="wss"
)

In [ ]:
import dask.distributed

client = dask.distributed.Client(cluster)
client

#### ☝️ Don’t forget to click the \"Dashboard\" link above to view the cluster dashboard!

### Load full dataset

Dask's `read_csv` functions makes it easy to read in all the CSV files in the dataset.

In [ ]:
import dask.dataframe as dd

# Load the entire dataset using Dask
ddf = dd.read_csv("s3://coiled-data/higgs/higgs-*.csv", storage_options={"anon": True})
ddf

Dask's machine learning library, [Dask-ML](https://ml.dask.org/), mimics Scikit-learn's API, providing scalable versions of functions of `sklearn.datasets.make_classification` and `sklearn.model_selection.train_test_split` that are designed to work with Dask Arrays and DataFrames in larger-than-memory settings.

Let's use Dask-ML to generate a similar classification dataset as before, but now with 100 million samples.

In [ ]:
from dask_ml.model_selection import train_test_split

X, y = ddf.iloc[:, 1:], ddf["labels"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=2)

Next we'll [persist our training and testing datasets](https://distributed.dask.org/en/latest/memory.html#persisting-collections) into distributed memory to avoid any unnecessary re-computations.

In [ ]:
import dask

X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

X_train

To do distributed training of an XGBoost model, we'll use the [Dask-XGBoost](https://github.com/dask/dask-xgboost) package which mirrors XGBoost's interface but works with Dask Arrays and DataFrames.

In [ ]:
import dask_xgboost

bst = dask_xgboost.train(client, params, X_train, y_train, num_boost_round=3)

Finally, we can again compute and plot the ROC curve for this model's predictions.

In [ ]:
y_pred = dask_xgboost.predict(client, bst, X_test)

y_test, y_pred = dask.compute(y_test, y_pred)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(fpr, tpr, lw=3,
        label='ROC Curve (area = {:.2f})'.format(auc(fpr, tpr)))
ax.plot([0, 1], [0, 1], "k--", lw=2)
ax.set(
    xlim=(0, 1),
    ylim=(0, 1),
    title="ROC Curve",
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
)
ax.legend()
plt.show()

Voilà! Congratulations on training a boosted decision tree in the cloud.

---

To run more computations like this, install Coiled locally with: `pip/conda install coiled` and check out other examples in the [coiled-resources repository](https://github.com/coiled/coiled-resources)!